<a href="https://colab.research.google.com/github/ZiyueNie/19ma573ZiyueNie/blob/master/src/hw8_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Price call for heston model **

By material in lecture,  we have
$$ \begin{align*}
&dS_{t} = rS_{t}dt + \sqrt{v_{t}}S_{t}dW_{1,t} \\
&dv_{t} = \kappa (\theta - v_{t})dt + \xi \sqrt{v_{t}} (\rho dW_{1,t} + \bar \rho dW_{2,t})
\end{align*}$$
<br>
$$C = S_{0}I_{1} - Ke^{-rT}I_{2}$$

where

$$I_{1}(\phi,\ln(K)) = \frac{1}{2} + \frac{1}{\pi}\int_{0}^{\infty}Re(\frac{e^{-iu\ln(K)}\phi(u-i)}{iu\phi(-i)})du$$

and
$$I_{2}(\phi,\ln(K)) = \frac{1}{2} + \frac{1}{\pi}\int_{0}^{\infty}Re(\frac{e^{-iu\ln(K)}\phi(u)}{iu})du$$

<br>By Ali Hirsa's book Computational Methods in Finance, Chapter 2.3.4.2, use $\xi$ in stead of $\sigma$, and let $q=0$, we have
$$ \begin{align*}
\phi(u) 
&= E(e^{iu\ln S_t}) \\
&=\frac{\exp({iu\ln S_{0} + iurt + \frac{\kappa \theta t (\kappa - i\rho\xi u)}{\xi^{2}}})} {(\cosh(\frac{\gamma t}{2})+ \frac{\kappa - i\rho\xi u}{\gamma}\sinh(\frac{\gamma t}{2}))^{\frac{2\kappa\theta}{\xi^{2}}}} * exp({\frac{-(u^{2}+iu)v_{0}} {\gamma coth(\frac{\gamma t}{2})+\kappa - i\rho\xi u}})
\end{align*} $$

Where $\gamma = \sqrt{\xi^{2}(u^{2}+iu) + (\kappa - i\rho\xi u)^{2}}$.



In [34]:
!git clone https://github.com/ZiyueNie/19ma573ZiyueNie

Cloning into '19ma573ZiyueNie'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 424 (delta 48), reused 0 (delta 0), pack-reused 301
Receiving objects: 100% (424/424), 585.09 KiB | 30.79 MiB/s, done.
Resolving deltas: 100% (170/170), done.


In [35]:
cd 19ma573ZiyueNie/src/

/content/19ma573ZiyueNie/src/19ma573ZiyueNie/src


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss
import scipy.optimize as so
import scipy.integrate as integrate
from contract_v01 import VanillaOption
from sde_1d_v01 import *

In [0]:
#paras 
s0=100
v0=0.04
r=0.05
kappa=1.2
theta=0.04
xi=0.3
rho=0.5
otype=1
K=100
T=1
market_price=15;

In [0]:
def Phi(u, s0, v0, T, r, kappa, theta, xi, rho):
  gamma = np.sqrt(((xi**2) * (u**2 + 1j*u)) + ((kappa - 1j*rho*xi*u)**2))  # 1j means i
  factor1 = np.exp((1j*u*np.log(s0)) + (1j*u*r*T) + ((kappa*theta*T*(kappa - (1j*rho*xi*u)))/ (xi**2)))
  factor2 = (np.cosh(gamma*T/2) + ((kappa-1j*rho*xi*u)/gamma)*np.sinh(gamma*T/2))**(2*kappa*theta/(xi**2))
  coth = np.cosh(gamma*T/2) / np.sinh(gamma*T/2) # since numpy has no attribute coth
  factor3 = np.exp(-(u**2 + 1j*u)*v0/(gamma*coth + kappa - 1j*rho*xi*u))
  return factor1 / factor2 * factor3

In [0]:
def Re1(u, s0, v0, T, r, kappa, theta, xi, rho):
  return (np.exp(-1j*np.log(K/s0)*u)*Phi(u-1j, s0, v0, T, r, kappa, theta, xi, rho)/(1j*u*Phi(-1j, s0, v0, T, r, kappa, theta, xi, rho))).real

In [0]:
def Re2(u, s0, v0, T, r, kappa, theta, xi, rho):
  return (np.exp(-1j*np.log(K/s0)*u)*Phi(u, s0, v0, T, r, kappa, theta, xi, rho)/(1j*u)).real

In [0]:
I1=1/2+(1/np.pi)*integrate.quad(lambda u:Re1(u, s0, v0, T, r, kappa, theta, xi, rho),0,1000)[0]
I2=1/2+(1/np.pi)*integrate.quad(lambda u:Re2(u, s0, v0, T, r, kappa, theta, xi, rho),0,1000)[0]

In [45]:
Price= (s0*I1)-(K*(np.exp(-r*T))*I2)
print('Fourier Transform of BSM price is '+ str(Price))

Fourier Transform of BSM price is 4.877057549989402


In [48]:
vanillaoption=VanillaOption(otype, K, T, market_price)
gbm=Gbm_1d(s0, r, xi*np.sqrt(v0))
price=gbm.bsm_price(vanillaoption)
print('european call option price is '+ str(price))

european call option price is 10.317923528899087
